In [1]:
!pip install regressors

  Stored in directory: /tmp/.cache/pip/wheels/b2/89/fc/7867f77234d0033395f7ad9814f245b337139acaa06b085aa2
Successfully built regressors
You are using pip version 19.0.3, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import numpy as np
import pandas as pd 
import statsmodels.formula.api as sm
import statsmodels.api as sma

from sklearn.linear_model import LinearRegression 
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold, cross_val_score,cross_val_predict, LeaveOneOut
from mlxtend.feature_selection import SequentialFeatureSelector as sfs
from sklearn.linear_model import LogisticRegression

import os
print(os.listdir("../input"))

['cardio_train.csv']


In [3]:
df = pd.read_csv('../input/cardio_train.csv',sep=';')
df = df.dropna()
df = df.drop(columns=['id'])
df.head()

,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,18857,1,165,64.0,130,70,3,1,0,0,0,1
3,17623,2,169,82.0,150,100,1,1,0,0,1,1
4,17474,1,156,56.0,100,60,1,1,0,0,0,0


In [4]:
df.corr()

,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
age,1.000000,-0.022811,-0.081515,0.053684,0.020764,0.017647,0.154424,0.098703,-0.047633,-0.029723,-0.009927,0.238159
gender,-0.022811,1.000000,0.499033,0.155406,0.006005,0.015254,-0.035821,-0.020491,0.338135,0.170966,0.005866,0.008109
height,-0.081515,0.499033,1.000000,0.290968,0.005488,0.006150,-0.050226,-0.018595,0.187989,0.094419,-0.006570,-0.010821
weight,0.053684,0.155406,0.290968,1.000000,0.030702,0.043710,0.141768,0.106857,0.067780,0.067113,-0.016867,0.181660
ap_hi,0.020764,0.006005,0.005488,0.030702,1.000000,0.016086,0.023778,0.011841,-0.000922,0.001408,-0.000033,0.054475
ap_lo,0.017647,0.015254,0.006150,0.043710,0.016086,1.000000,0.024019,0.010806,0.005186,0.010601,0.004780,0.065719
cholesterol,0.154424,-0.035821,-0.050226,0.141768,0.023778,0.024019,1.000000,0.451578,0.010354,0.035760,0.009911,0.221147
gluc,0.098703,-0.020491,-0.018595,0.106857,0.011841,0.010806,0.451578,1.000000,-0.004756,0.011246,-0.006770,0.089307
smoke,-0.047633,0.338135,0.187989,0.067780,-0.000922,0.005186,0.010354,-0.004756,1.000000,0.340094,0.025858,-0.015486
alco,-0.029723,0.170966,0.094419,0.067113,0.001408,0.010601,0.035760,0.011246,0.340094,1.000000,0.025476,-0.007330


**Part A - Interactions between variables**

According to the correlation in the data, we picked 6 predictors for creating interaction models with logistic regression.

**Top Six Predictors**
- Age
- cholesterol
- Weight
- ap_lo
- ap_hi
- active


**Pseudo R-squareds**

"When analyzing data with a logistic regression, an equivalent statistic to R-squared does not exist.  The model estimates from a logistic regression are maximum likelihood estimates arrived at through an iterative process.  They are not calculated to minimize variance, so the OLS approach to goodness-of-fit does not apply.  However, to evaluate the goodness-of-fit of logistic models, several pseudo R-squareds have been developed.   These are “pseudo” R-squareds because they look like R-squared in the sense that they are on a similar scale, ranging from 0 to 1 (though some pseudo R-squareds never achieve 0 or 1) with higher values indicating better model fit, but they cannot be interpreted as one would interpret an OLS R-squared and different pseudo R-squareds can arrive at very different values."[1]

[1] https://stats.idre.ucla.edu/other/mult-pkg/faq/general/faq-what-are-pseudo-r-squareds/

    1. Interaction model with age and cholesterol

In [5]:
f1 = 'cardio ~ age*cholesterol'
logitfit = sm.logit(formula = str(f1), data = df).fit()
print(logitfit.summary())

Optimization terminated successfully.
         Current function value: 0.644303
         Iterations 5
                           Logit Regression Results                           
Dep. Variable:                 cardio   No. Observations:                70000
Model:                          Logit   Df Residuals:                    69996
Method:                           MLE   Df Model:                            3
Date:                Wed, 08 May 2019   Pseudo R-squ.:                 0.07047
Time:                        22:38:45   Log-Likelihood:                -45101.
converged:                       True   LL-Null:                       -48520.
                                        LLR p-value:                     0.000
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          -5.9629      0.157    -37.970      0.000      -6.271      -5.655
age           


The p-values in the output below tell us that the interaction effect (age*cholesterol) is statistically significant. 

2. Interaction model with age and weight

In [6]:
f2 = 'cardio ~ age*weight'
logitfit2 = sm.logit(formula = str(f2), data = df).fit()
print(logitfit2.summary())

Optimization terminated successfully.
         Current function value: 0.647814
         Iterations 5
                           Logit Regression Results                           
Dep. Variable:                 cardio   No. Observations:                70000
Model:                          Logit   Df Residuals:                    69996
Method:                           MLE   Df Model:                            3
Date:                Wed, 08 May 2019   Pseudo R-squ.:                 0.06540
Time:                        22:38:46   Log-Likelihood:                -45347.
converged:                       True   LL-Null:                       -48520.
                                        LLR p-value:                     0.000
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -9.5410      0.356    -26.777      0.000     -10.239      -8.843
age            0.0004   1.81e


The p-values in the output below tell us that the interaction effect (age*weight) is statistically significant. 

3. Interaction model with Weight, Cholesterol, and Age

In [7]:

f3 = 'cardio ~ weight*cholesterol*age'
logitfit3 = sm.logit(formula = str(f3), data = df).fit()
print(logitfit3.summary())


Optimization terminated successfully.
         Current function value: 0.631603
         Iterations 5
                           Logit Regression Results                           
Dep. Variable:                 cardio   No. Observations:                70000
Model:                          Logit   Df Residuals:                    69992
Method:                           MLE   Df Model:                            7
Date:                Wed, 08 May 2019   Pseudo R-squ.:                 0.08879
Time:                        22:38:46   Log-Likelihood:                -44212.
converged:                       True   LL-Null:                       -48520.
                                        LLR p-value:                     0.000
                             coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                -10.5819      0.859    -12.326      0.000     -12.265      

The summary indicates that adding an interaction of weight, cholesterol, and age does not improve the yield of the model. The interaction is insignificant.

4. Interaction model with Age and Ap_hi

In [8]:
f4 = 'cardio ~ age*ap_hi'
logitfit4 = sm.logit(formula = str(f4), data = df).fit()
print(logitfit4.summary())

Optimization terminated successfully.
         Current function value: 0.600411
         Iterations 8
                           Logit Regression Results                           
Dep. Variable:                 cardio   No. Observations:                70000
Model:                          Logit   Df Residuals:                    69996
Method:                           MLE   Df Model:                            3
Date:                Wed, 08 May 2019   Pseudo R-squ.:                  0.1338
Time:                        22:38:46   Log-Likelihood:                -42029.
converged:                       True   LL-Null:                       -48520.
                                        LLR p-value:                     0.000
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    -29.3734      0.645    -45.523      0.000     -30.638     -28.109
age            0.0012   3.19e

The p-values in the output below tell us that the interaction effect (age*ap_hi) is statistically significant. 

5. Interaction model with Weight and Active

In [9]:
f5 = 'cardio ~ weight*active'
logitfit5 = sm.logit(formula = str(f5), data = df).fit()
print(logitfit5.summary())

Optimization terminated successfully.
         Current function value: 0.675607
         Iterations 5
                           Logit Regression Results                           
Dep. Variable:                 cardio   No. Observations:                70000
Model:                          Logit   Df Residuals:                    69996
Method:                           MLE   Df Model:                            3
Date:                Wed, 08 May 2019   Pseudo R-squ.:                 0.02531
Time:                        22:38:46   Log-Likelihood:                -47292.
converged:                       True   LL-Null:                       -48520.
                                        LLR p-value:                     0.000
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept        -1.8929      0.097    -19.512      0.000      -2.083      -1.703
weight            0.

The summary indicates that weight and active does not have strong interaction. The P-values demonstrate insignificance.

6. Interaction model with Age, Cholesterol, Weight, and Ap_hi

In [10]:
f6 = 'cardio ~ age*cholesterol*weight*ap_hi'
logitfit6 = sm.logit(formula = str(f6), data = df).fit()
print(logitfit6.summary())

Optimization terminated successfully.
         Current function value: 0.581240
         Iterations 9
                           Logit Regression Results                           
Dep. Variable:                 cardio   No. Observations:                70000
Model:                          Logit   Df Residuals:                    69985
Method:                           MLE   Df Model:                           14
Date:                Wed, 08 May 2019   Pseudo R-squ.:                  0.1614
Time:                        22:38:47   Log-Likelihood:                -40687.
converged:                       True   LL-Null:                       -48520.
                                        LLR p-value:                     0.000
                                   coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------
Intercept                       22.5506      6.684      3.374      0.001

**The p-values and Pseudo R-squ. in the output above tells us that the interaction effect is statistically significant. This might be one of the candidates for the final model.**

**Part B - Numerical Transformations**

Explore Polynomial (up to 3rd order or Cubic) or Logarithmic transformations of at least two variables. Determine if these transformations are significant. 

In [11]:
#Polynomial
fPolynomial = 'cardio ~ age+I(age*age)+I(age*age*age)'
logitfitPolynomial = sm.logit(formula = str(fPolynomial), data = df).fit()
print(logitfitPolynomial.summary())

Optimization terminated successfully.
         Current function value: 0.663352
         Iterations 5
                           Logit Regression Results                           
Dep. Variable:                 cardio   No. Observations:                70000
Model:                          Logit   Df Residuals:                    69997
Method:                           MLE   Df Model:                            2
Date:                Wed, 08 May 2019   Pseudo R-squ.:                 0.04298
Time:                        22:38:47   Log-Likelihood:                -46435.
converged:                       True   LL-Null:                       -48520.
                                        LLR p-value:                     0.000
                         coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept            -41.0686      3.703    -11.092      0.000     -48.326     -33.812
age  

- **Summary:** Polynomial transformations does not demonstrate significant improvements to the model as the outcome of the model is binary.

In [12]:
#Logarithmic
fLog = 'cardio ~ np.log(age)+np.log(ap_hi)'
logitfitLog = sm.logit(formula = str(fLog), data = df).fit()
print(logitfitLog.summary())

<string>:1: RuntimeWarning: invalid value encountered in log
<string>:1: RuntimeWarning: invalid value encountered in log


Optimization terminated successfully.
         Current function value: 0.596972
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:                 cardio   No. Observations:                69993
Model:                          Logit   Df Residuals:                    69990
Method:                           MLE   Df Model:                            2
Date:                Wed, 08 May 2019   Pseudo R-squ.:                  0.1388
Time:                        22:38:47   Log-Likelihood:                -41784.
converged:                       True   LL-Null:                       -48515.
                                        LLR p-value:                     0.000
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept       -61.2244      0.730    -83.892      0.000     -62.655     -59.794
np.log(age)       2.

- **Summary:** Logarithmic transformations demonstrate improvements to the model as seen through the resulting P-values and other summary attributes. This might be one of the candidates for the final model. 

**Part C - Forward Model Selection**

In [13]:
def forward_selected(inputDF, outputDF, df):
    highestScore = 0.0
    selected = []
    outputColName = str(outputDF.columns[0])
    
    for x in range(2, len(inputDF.columns)+1):
        model = sfs(LinearRegression(),k_features=x,forward=True,verbose=2,cv=5,n_jobs=-1,scoring='r2')
        model.fit(inputDF,outputDF)
        predictors = model.k_feature_names_
        formula = "{} ~ {}".format(outputColName, ' + '.join(predictors))
        candidateModel = sm.logit(formula, df).fit()
        
        if(candidateModel.prsquared > highestScore):
            selected = predictors
            highestScore = candidateModel.prsquared
            print("score:", candidateModel.prsquared)
    
    formula = "{} ~ {}".format(outputColName,' + '.join(selected))
    model = sm.logit(formula, df).fit()
    return model
    

In [14]:
inputDF = df[["age", "gender", "height", "weight", "ap_hi", "ap_lo","cholesterol","gluc","smoke","alco","active"]]
outputDF = df[["cardio"]]
result = forward_selected(inputDF, outputDF, df)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  11 out of  11 | elapsed:    5.0s finished

[2019-05-08 22:38:53] Features: 1/2 -- score: 0.056663329022313344[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    1.6s finished

[2019-05-08 22:38:54] Features: 2/2 -- score: 0.09146490520866582

Optimization terminated successfully.
         Current function value: 0.645167
         Iterations 5
score: 0.06922041315629845


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of  11 | elapsed:    0.3s remaining:    0.6s
[Parallel(n_jobs=-1)]: Done  11 out of  11 | elapsed:    0.7s finished

[2019-05-08 22:38:55] Features: 1/3 -- score: 0.056663329022313344[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    1.0s finished

[2019-05-08 22:38:56] Features: 2/3 -- score: 0.09146490520866582[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   7 out of   9 | elapsed:    0.9s remaining:    0.3s
[Parallel(n_jobs=-1)]: Done   9 out of   9 | elapsed:    1.0s finished

[2019-05-08 22:38:57] Features: 3/3 -- score: 0.11253233390390807

Optimization terminated successfully.
         Current function value: 0.633309
         Iterations 5
score: 0.08632781025389646


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  11 out of  11 | elapsed:    0.7s finished

[2019-05-08 22:38:58] Features: 1/4 -- score: 0.056663329022313344[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    1.0s finished

[2019-05-08 22:38:59] Features: 2/4 -- score: 0.09146490520866582[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   7 out of   9 | elapsed:    0.9s remaining:    0.3s
[Parallel(n_jobs=-1)]: Done   9 out of   9 | elapsed:    1.0s finished

[2019-05-08 22:39:00] Features: 3/4 -- score: 0.11253233390390807[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   6 out of   8 | elapsed:    1.0s remaining:    0.3s
[Parallel(n_jobs=-1)]: Done   8 out of   8 | elapsed:    1.0s finished

[2019-05-08 22:39:01] Features: 4/4 -- score: 0.11

...

Optimization terminated successfully.
         Current function value: 0.592133
         Iterations 7
score: 0.14573285028383698


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of  11 | elapsed:    0.3s remaining:    0.5s
[Parallel(n_jobs=-1)]: Done  11 out of  11 | elapsed:    0.7s finished

[2019-05-08 22:39:32] Features: 1/10 -- score: 0.056663329022313344[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.9s finished

[2019-05-08 22:39:33] Features: 2/10 -- score: 0.09146490520866582[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   7 out of   9 | elapsed:    0.7s remaining:    0.2s
[Parallel(n_jobs=-1)]: Done   9 out of   9 | elapsed:    0.8s finished

[2019-05-08 22:39:34] Features: 3/10 -- score: 0.11253233390390807[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   6 out of   8 | elapsed:    0.8s remaining:    0.3s
[Parallel(n_jobs=-1)]: Done   8 out of

Optimization terminated successfully.
         Current function value: 0.592132
         Iterations 7
score: 0.14573377940523813


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  11 out of  11 | elapsed:    0.7s finished

[2019-05-08 22:39:39] Features: 1/11 -- score: 0.056663329022313344[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.9s finished

[2019-05-08 22:39:40] Features: 2/11 -- score: 0.09146490520866582[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   7 out of   9 | elapsed:    0.7s remaining:    0.2s
[Parallel(n_jobs=-1)]: Done   9 out of   9 | elapsed:    0.8s finished

[2019-05-08 22:39:41] Features: 3/11 -- score: 0.11253233390390807[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   6 out of   8 | elapsed:    0.7s remaining:    0.2s
[Parallel(n_jobs=-1)]: Done   8 out of   8 | elapsed:    0.8s finished

[2019-05-08 22:39:42] Features: 4/11 -- score: 

Optimization terminated successfully.
         Current function value: 0.629393
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.592132
         Iterations 7


In [15]:
print(result.model.formula)
print("Selected Model:", result.model.formula)
print("Selected Pseudo R-squqre:", result.prsquared)
print(result.summary())

cardio ~ age + gender + height + weight + ap_hi + ap_lo + cholesterol + gluc + alco + active
Selected Model: cardio ~ age + gender + height + weight + ap_hi + ap_lo + cholesterol + gluc + alco + active
Selected Pseudo R-squqre: 0.14573377940523813
                           Logit Regression Results                           
Dep. Variable:                 cardio   No. Observations:                70000
Model:                          Logit   Df Residuals:                    69989
Method:                           MLE   Df Model:                           10
Date:                Wed, 08 May 2019   Pseudo R-squ.:                  0.1457
Time:                        22:39:44   Log-Likelihood:                -41449.
converged:                       True   LL-Null:                       -48520.
                                        LLR p-value:                     0.000
                  coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------

By doing forward model selection, 'cardio ~ age + gender + height + weight + ap_hi + ap_lo + cholesterol + gluc + smoke + alco + active' became a candidate of the final model.

**Part D - Backward Model Selection**

In [16]:
def backward_selected(inputDF, outputDF, df):
    highestScore = 0.0
    selected = []
    outputColName = str(outputDF.columns[0])
    for x in range(2, len(inputDF.columns)+1):
        model = sfs(LinearRegression(),k_features=x,forward=False,verbose=2,cv=5,n_jobs=-1,scoring='r2')
        model.fit(inputDF,outputDF)
        predictors = model.k_feature_names_
        formula = "{} ~ {}".format(outputColName, ' + '.join(predictors))
        candidateModel = sm.logit(formula, df).fit()
        
        if(candidateModel.prsquared > highestScore):
            selected = predictors
            highestScore = candidateModel.prsquared
            print("score:", candidateModel.prsquared)
    
    formula = "{} ~ {}".format(outputColName,' + '.join(selected))
    model = sm.logit(formula, df).fit()
    return model
      

In [17]:
inputDF = df[["age", "gender", "height", "weight", "ap_hi", "ap_lo","cholesterol","gluc","smoke","alco","active"]]
outputDF = df[["cardio"]]
result = backward_selected(inputDF, outputDF, df)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  11 out of  11 | elapsed:    5.6s finished

[2019-05-08 22:39:50] Features: 10/2 -- score: 0.11857435621992156[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    1.9s finished

[2019-05-08 22:39:52] Features: 9/2 -- score: 0.1184418783102366[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   7 out of   9 | elapsed:    1.5s remaining:    0.4s
[Parallel(n_jobs=-1)]: Done   9 out of   9 | elapsed:    1.7s finished

[2019-05-08 22:39:54] Features: 8/2 -- score: 0.11828868161011627[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   6 out of   8 | elapsed:    1.1s remaining:    0.4s
[Parallel(n_jobs=-1)]: Done   8 out of   8 | elapsed:    1.2s finished

[2019-05-08 22:39:55] Features: 7/2 -- score: 0.117

Optimization terminated successfully.
         Current function value: 0.628996
         Iterations 6
score: 0.09254987814185744


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  11 out of  11 | elapsed:    5.6s finished

[2019-05-08 22:40:03] Features: 10/3 -- score: 0.11857435621992156[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    2.0s finished

[2019-05-08 22:40:05] Features: 9/3 -- score: 0.1184418783102366[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   7 out of   9 | elapsed:    0.8s remaining:    0.2s

STOPPING EARLY DUE TO KEYBOARD INTERRUPT...

Optimization terminated successfully.
         Current function value: 0.592133
         Iterations 7
score: 0.14573285028383698


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.

STOPPING EARLY DUE TO KEYBOARD INTERRUPT...

...

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.

STOPPING EARLY DUE TO KEYBOARD INTERRUPT...

Optimization terminated successfully.
         Current function value: 0.592019
         Iterations 7


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   6 out of   8 | elapsed:    2.1s remaining:    0.7s

STOPPING EARLY DUE TO KEYBOARD INTERRUPT...ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2963, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-17-9dd3afc12f18>", line 3, in <module>
    result = backward_selected(inputDF, outputDF, df)
  File "<ipython-input-16-56539e06f3ce>", line 10, in backward_selected
    candidateModel = sm.logit(formula, df).fit()
  File "/opt/conda/lib/python3.6/site-packages/statsmodels/discrete/discrete_model.py", line 1834, in fit
    disp=disp, callback=callback, **kwargs)
  File "/opt/conda/lib/python3.6/site-packages/statsmodels/discrete/discrete_model.py", line 220, in fit
    disp=disp, callback=callback, **kwargs)
  File "/opt/conda/lib/python3.6/site-packages/statsmodels/base/model.py", line 466, in fit
    full_output=full_output)
  File "/opt/conda/lib/python3.6/site-packages/statsmodels/base/optimizer.py", line 191, in _fit
    hess=hessian)
  File "/opt/conda/lib/python3.6/s

KeyboardInterrupt: 

In [ ]:
print("Selected Model:", result.model.formula)
print("Selected Pseudo R-squqre:", result.prsquared)
print(result.summary())

By doing forward model selection, 'cardio ~ age + gender + height + weight + ap_hi + ap_lo + cholesterol + gluc + smoke + alco + active' became a candidate of the final model.

**Analysis:** As forward selection and backward selection have the same output, we can summerize that the possible final model of the problem is 'cardio ~ age + gender + height + weight + ap_hi + ap_lo + cholesterol + gluc + smoke + alco + active'. However, the P-Value of gender in this model is higher than 0.05 so we have tried removing it out from the model.   

In [ ]:
#Model without Gender
formula = "cardio ~ age + height + weight + ap_hi + ap_lo + cholesterol + gluc + smoke + alco + active"
model = sm.logit(formula, df).fit()
print(model.summary())

**As you can see, by removing gender doesn't negatively affect the model. Thus, "cardio ~ age + height + weight + ap_hi + ap_lo + cholesterol + gluc + smoke + alco + active" became a candidate of the final model.**

**Part D-2  Decision Tree**

Because this problem is a kind of classification problem, Decision Tree might be used to create a model.

In [ ]:
!pip install pygam

In [ ]:
!pip install pydotplus

In [ ]:
from IPython.display import Image 
import pydotplus  

In [ ]:
from sklearn.tree import DecisionTreeClassifier

inputDF_DC = df[["age", "gender", "height", "weight", "ap_hi", "ap_lo","cholesterol","gluc","smoke","alco","active"]]
outputDF_DC = df[["cardio"]]

X_train_DC, X_test_DC, y_train_DC, y_test_DC = train_test_split(inputDF_DC, outputDF_DC, test_size=0.30)

DTR = DecisionTreeClassifier()
clfDC = DTR.fit(X_train_DC, y_train_DC)


In [ ]:
clfDC = DecisionTreeClassifier(random_state = 0)

clfDC = clfDC.fit(X_train_DC,y_train_DC)

**Part E - Cross Validation**

**As our problem is a part of classification, accuracy will be mesuared instead of the RMSE. "Accuracy" shows how many times a model predicted the right value/ class among all data set.**

**Moreover, the data input is too huge to use Leave One Out Classification as it takes very long time to process. Thus, K-Fold cross validation will be used instead**

In [28]:
from sklearn.model_selection import cross_val_score, KFold, LeaveOneOut
from sklearn.linear_model import LogisticRegression
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.preprocessing import PolynomialFeatures
from patsy import dmatrices

In [20]:
#Create Input
inputDF = df[["age", "gender", "height", "weight", "ap_hi", "ap_lo","cholesterol","gluc","smoke","alco","active"]]
outputDF = df[["cardio"]]

1. Cross Validation for cardio ~ age * cholesterol * weight * ap_hi

In [32]:
# create dummy variables, and their interactions
outputDFInteraction, inputDFInteraction = dmatrices('cardio ~ age*cholesterol*weight*ap_hi', df, return_type="dataframe")
# flatten y into a 1-D array so scikit-learn can understand it
outputDFInteraction = np.ravel(outputDFInteraction)

In [33]:
logisticRegr = LogisticRegression()
kf = KFold(5, shuffle=True, random_state=42).get_n_splits(inputDFInteraction)
score = cross_val_score(logisticRegr, inputDFInteraction, outputDFInteraction, scoring="accuracy", cv = kf)
print("Score of cardio ~ age*cholesterol*weight*ap_hi:", score.mean())

/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  Futur

Score of cardio ~ age*cholesterol*weight*ap_hi: 0.7137999392043


2. Cross Validation for "cardio ~ np.log(age)+np.log(ap_hi)"

In [34]:
#Create Input
inputDFLog = df[["age", "ap_hi","cardio"]]
inputDFLog["age"] = np.log(inputDFLog["age"])
inputDFLog["ap_hi"] = np.log(inputDFLog["ap_hi"])
inputDFLog = inputDFLog.dropna()

outputDFLog = inputDFLog["cardio"]
inputDFLog = inputDFLog.drop(columns=['cardio'])

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in log
  after removing the cwd from sys.path.
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [35]:
logisticRegr = LogisticRegression()
kf = KFold(5, shuffle=True, random_state=42).get_n_splits(inputDFLog)
score = cross_val_score(logisticRegr, inputDFLog, outputDFLog, scoring="accuracy", cv = kf)
print("Score of cardio ~ np.log(age)+np.log(ap_hi):", score.mean())

/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Score of cardio ~ np.log(age)+np.log(ap_hi): 0.7132427237393449


/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


3. Cross Validation for "cardio ~ age + height + weight + ap_hi + ap_lo + cholesterol + gluc + smoke + alco + active"

In [25]:
#Create Input
inputDF = df[["age", "gender", "height", "weight", "ap_hi", "ap_lo","cholesterol","gluc","smoke","alco","active"]]
outputDF = df[["cardio"]]


In [26]:
logisticRegr = LogisticRegression()
kf = KFold(5, shuffle=True, random_state=42).get_n_splits(inputDF)
score = cross_val_score(logisticRegr, inputDF, outputDF, scoring="accuracy", cv = kf)
print("Score of cardio ~ age + height + weight + ap_hi + ap_lo + cholesterol + gluc + smoke + alco + active:", score.mean())

/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:43

Score of cardio ~ age + height + weight + ap_hi + ap_lo + cholesterol + gluc + smoke + alco + active: 0.7066715065340383


4. Accuracy score of Decision Tree

We don't need to do cross validaiton in the decision tree model as the model does not have coefficients like logistic regression model.

In [ ]:
y_pred_DC = clfDC.predict(X_test_DC)

print("Accuracy:",metrics.accuracy_score(y_test_DC, y_pred_DC))

**Part F- Final Model**

1. Based on the findings from the interactions, forward, backward model selection and decision trees, we had many candidates for the final model. During the analysis of the interactions, we found the best model was cardio ~ age*cholesterol*weight*ap_hi. For forward and backward model selection, both resulted with this model: cardio ~ age + gender + height + weight + ap_hi + ap_lo + cholesterol + gluc + smoke + alco + active. Analyzing further, we noticed that we could improve this model by removing 'gender'. We then performed accuracy tests to receives scores for all of the model candidates including decision tree analysis. While the results were fairly close, the final model with the best score is 'cardio ~ age*cholesterol*weight*ap_hi.


2. The model was not what we expected based on our original understanding of the data. At an initial glance, we thought our final model would include the majority of the variables since we assumed they all would have an effect on the cardiovascular outcome. We were surprised to find our various model candidates had pretty similar accuracy scores yet were fairly different types of models. 


3. Based on the new model, we would not make any changes to our validation strategy proposed in our previous project deliverable. While removing certain variables from the user interface of our proposed tool could be done, there is no need to remove them as it serves as additional patient information for doctors and practictioners. These could also prove to be useful for other models in the future especially since our final candidates had similar accuracy results. We would proceed with having doctors validate whether or not cardiovascular disease is seen in their patients to help improve the tool. 